# $t$-NMF ($t$ Non-negative Matrix Factorization)

## Factorization

In [ ]:
%%shell
git clone https://github.com/tky823/audio_source_separation.git

In [ ]:
%cd "/content/audio_source_separation/egs/nmf-example/t-nmf"

In [ ]:
import sys
sys.path.append("../../../src")

In [ ]:
import numpy as np
import scipy.signal as ss
import soundfile as sf
import IPython.display as ipd
import matplotlib.pyplot as plt

In [ ]:
from algorithm.nmf import tNMF

In [ ]:
plt.rcParams['figure.dpi'] = 200

In [ ]:
fft_size, hop_size = 1024, 256

In [ ]:
x, sr = sf.read("../../../dataset/sample-song/sample-1_electric-guitar_8000.wav")

### Original Source

In [ ]:
ipd.Audio(x, rate=sr)

In [ ]:
_, _, spectrogram = ss.stft(x, nperseg=fft_size, noverlap=fft_size-hop_size)
X = np.abs(spectrogram)**2

In [ ]:
n_basis = 6

### Execution by $t$-NMF

In [ ]:
np.random.seed(111)
nmf = tNMF(n_basis=n_basis)

In [ ]:
basis, activation = nmf(X, iteration=100)

In [ ]:
plt.figure()
plt.plot(nmf.loss, color='black')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.show()

In [ ]:
X[X < 1e-12] = 1e-12

### Recostructed Signal

In [ ]:
X[X < 1e-12] = 1e-12

In [ ]:
Z = basis @ activation

ratio = np.sqrt(Z / X)

estimated_spectrogram = ratio * spectrogram
_, estimated_signal = ss.istft(estimated_spectrogram, nperseg=fft_size, noverlap=fft_size-hop_size)
estimated_signal = estimated_signal / np.abs(estimated_signal).max()
ipd.Audio(estimated_signal, rate=sr)

### Factorized Signals

In [ ]:
for idx in range(n_basis):
    Z = basis[:, idx: idx+1] * activation[idx: idx+1, :]

    ratio = np.sqrt(Z / X)

    estimated_spectrogram = ratio * spectrogram
    _, estimated_signal = ss.istft(estimated_spectrogram, nperseg=fft_size, noverlap=fft_size-hop_size)
    estimated_signal = estimated_signal / np.abs(estimated_signal).max()
    display(ipd.Audio(estimated_signal, rate=sr))

In [ ]:
for idx in range(n_basis):
    estimated_spectrogram = basis[:, idx: idx + 1] @ activation[idx: idx + 1, :]

    estimated_power = np.abs(estimated_spectrogram)**2
    estimated_power[estimated_power < 1e-12] = 1e-12
    log_spectrogram = 10 * np.log10(estimated_power)

    plt.figure()
    plt.pcolormesh(log_spectrogram, cmap='jet')
    plt.show()

## Degree of Freedom Parameter $\nu$

In [ ]:
np.random.seed(111)
nmf_nu1e0 = tNMF(n_basis=n_basis, nu=10**0)
basis, activation = nmf_nu1e0(X, iteration=100)

In [ ]:
np.random.seed(111)
nmf_nu1e2 = tNMF(n_basis=n_basis, nu=1e+2)
basis, activation = nmf_nu1e2(X, iteration=100)

In [ ]:
np.random.seed(111)
nmf_nu1e4 = tNMF(n_basis=n_basis, nu=1e+4)
basis, activation = nmf_nu1e4(X, iteration=100)

In [ ]:
plt.figure()
plt.plot(nmf_nu1e0.loss, color='mediumvioletred', label=r'$\nu=10^{0}$')
plt.plot(nmf_nu1e2.loss, color='mediumblue', label=r'$\nu=10^{2}$')
plt.plot(nmf_nu1e4.loss, color='mediumseagreen', label=r'$\nu=10^{4}$')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.legend()
plt.show()